# PyTorch 常用命令汇总
[PyTorch Tutorial](https://pytorch.org/tutorials/)

[PyTorch Python API](https://pytorch.org/docs/stable/torch.html)

In [56]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Torch.tensor类矩阵数据

### 创建矩阵

#### 直接创建

In [2]:
x = torch.empty(5, 3)
x = torch.rand(5, 3)
x = torch.zeros(5, 3, dtype=torch.long)

In [ ]:
x = x.new_ones(5, 3, dtype=torch.double)
x = torch.randn_like(x, dtype=torch.float)
x = torch.tensor([5.5, 3])

#### 与其他模块的联动

In [ ]:
a = x.item()
b = x.numpy()
c = np.ones(5)
d = torch.from_numpy(c)

### 矩阵基本信息

In [6]:
print(x.size())

torch.Size([5, 3])


### 矩阵数据的处理

In [23]:
x = torch.rand(5, 3)
y = torch.rand(5, 3)

#### 矩阵运算

In [ ]:
print(x + y)
print(torch.add(x, y))
result = torch.empty(5, 3)
torch.add(x, y, out=result)

#### 矩阵重构

In [39]:
print(x.view(2,3))  #按指定形状不替换重构

tensor([[1.4906, 1.6711, 2.0519],
        [1.2254, 1.1406, 2.9968]])


## 神经网络解算流程

### 神经网络的搭建

#### 单层网络定义

In [3]:
fc1 = nn.Linear(5, 3, bias=True) #产生5输入、3输出的线性层，默认包含偏差（bias项可缺省）
conv1 = nn.Conv2d(3, 6, 5) #产生3输入、6输出、5*5大小的convolution层

#### 计算图定义

In [4]:
x=torch.randn(6,4)
x.unsqueeze_(0)  #为单个数据添加Batch层，以符合输入维度
x = F.relu(x) #Relu层
x = F.max_pool2d(x, (2, 2)) #最大值Pool层，2*2区域

In [5]:
print(x.size())

torch.Size([1, 3, 2])


#### 神经网络nn.Module 容器
  建立一个神经网络，以ReNet的建立为例

In [12]:
class Net(nn.Module):

    def __init__(self):
        """定义不同容器."""
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        """定义前向传播流程."""
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [13]:
net = Net()  #生成定义好的Net对象

#### 代价函数定义

In [60]:
criterion = nn.MSELoss()  #定义MSE均方损失容器
criterion_crossentropy = F.CrossEntropyLoss()

AttributeError: module 'torch.nn.functional' has no attribute 'CrossEntropyLoss'

#### 运算器定义
引用搭建好模型中的参数构建运算器

In [57]:
optimizer = optim.SGD(net.parameters(), lr=0.01)  #定义SGD运算器

### 神经网络求解

#### 参数初始化

In [58]:
net.zero_grad()  #所有参数梯度归零
optimizer.zero_grad()  #定义了运算器之后，直接将运算器的参数缓存归零

#### 前向传播

In [45]:
input = torch.randn(1, 1, 32, 32)
target = torch.randn(10)
target = target.view(1, -1)

out = net(input) #完成一次完整前向传播
loss = criterion(out, target)  # 计算代价

#### 后向传播

In [46]:
grad_randn = torch.randn(1, 10)
loss.backward() #完成一次完整前向传播
# out.backward(grad_randn)  #从特定层开始后向传播，注意每个参数一次前传后只能后传一次

#### 获取参数

In [55]:
params=list(net.parameters())  #获取全部参数列表
print(net.conv2.weight[15])  # 获取指定层指定参数，例：conv1层第16神经元参数
print(net.conv2.bias.grad)  # 获取指定层指定梯度，例：conv1层偏差项梯度

tensor([[[-9.7367e-02, -2.7092e-02,  9.1190e-03],
         [ 7.6643e-02,  1.2557e-01,  6.9187e-02],
         [-6.0590e-02,  7.4422e-02, -1.3327e-01]],

        [[ 1.0864e-01,  1.1039e-01,  9.6965e-02],
         [-3.0777e-02, -2.8466e-02,  9.5951e-02],
         [ 2.6086e-02, -1.0719e-01, -7.2378e-02]],

        [[ 5.0360e-02, -3.4538e-02,  1.0904e-01],
         [ 1.0685e-01, -1.3312e-01,  2.5779e-02],
         [-3.6451e-03, -9.2713e-02, -1.0369e-01]],

        [[ 2.6505e-02, -6.0805e-02, -8.4293e-03],
         [-9.1020e-02,  2.7264e-02,  5.1203e-02],
         [ 8.6776e-02,  1.0141e-01, -1.2973e-01]],

        [[-1.2748e-01,  8.1006e-02,  1.0380e-02],
         [-1.2379e-01,  9.0663e-02,  3.2367e-02],
         [ 2.0242e-02,  6.1821e-02, -5.9500e-02]],

        [[-5.6891e-03,  9.6778e-02,  6.2581e-02],
         [ 1.2645e-01, -9.5265e-02,  1.0756e-01],
         [ 7.4238e-05,  4.4013e-02,  8.6604e-02]]], grad_fn=<SelectBackward>)
tensor([ 0.1099, -0.0925,  0.0611,  0.1524, -0.2210,  0.0813, 

#### 升级参数

In [59]:
optimizer.step()  #单步优化